In [1]:
from tensorflow.io import read_file
from tensorflow.image import decode_jpeg, convert_image_dtype, resize
from tensorflow import reshape
import tensorflow as tf
import json

In [2]:
def read_json(json_file_path):

    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return data

In [4]:
def get_image_data(data_dict, base_img_dir = '"Data/ship/'):
    '''
        Implement the function to get the image path and corresponding camera transformation matrix.
    '''

    img_paths = []
    img_cam_transform_matrices = []

    all_frames = data_dict["frames"]


    return

In [6]:
class Image():
	def __init__(self, imageWidth, imageHeight):

		self.imageWidth = imageWidth
		self.imageHeight = imageHeight

	def __call__(self, imagePath):

		image = read_file(imagePath)
		image = decode_jpeg(image, 3)
		image = convert_image_dtype(image, dtype=tf.float32)
		image = resize(image, (self.imageWidth, self.imageHeight))
		image = reshape(image, (self.imageWidth, self.imageHeight, 3))

		return image